### Import das Bibliotecas

In [1]:
import os
import re
import librosa
import librosa.display

from sklearn.cluster import KMeans

import time
import pandas as pd
import numpy as np
import soundfile as sf

from itertools import combinations 

import IPython.display as ipd1
import matplotlib.pyplot as plt

from pathlib import Path

### Definição de Constantes

In [2]:
FFT_FRAME_SIZE = 2048
HOP_LENGTH = 1024

### Carregamento do Dataframe com os Diretórios dos Áudios

In [3]:
dsPath = '.\\Datasets\\Projeto\\'
dsInfo = pd.read_csv(dsPath+'DiretoriosEventos.csv',sep=';') 
dsInfo

,Diretorio,Evento
0,BoltAction22,Disparo Bolt
1,CracklingFire,Incêndio
2,GlassBreaking,Vidro Quebrando
3,WaterDrops,Água Pingando


### Inclusão da Classes Combinadas para Treinamento Multi Label

In [4]:
tamDF = len(dsInfo['Diretorio'])
dsInfoTmp = dsInfo['Diretorio']

count = 0

for x in range(2, tamDF+1):
    combos = combinations(dsInfoTmp, x)
    for comb in list(combos):
        print(['_'.join(comb), ''])
        dsInfo.loc[tamDF+count] = ['_'.join(comb), '']
        count = count + 1

['BoltAction22_CracklingFire', '']
['BoltAction22_GlassBreaking', '']
['BoltAction22_WaterDrops', '']
['CracklingFire_GlassBreaking', '']
['CracklingFire_WaterDrops', '']
['GlassBreaking_WaterDrops', '']
['BoltAction22_CracklingFire_GlassBreaking', '']
['BoltAction22_CracklingFire_WaterDrops', '']
['BoltAction22_GlassBreaking_WaterDrops', '']
['CracklingFire_GlassBreaking_WaterDrops', '']
['BoltAction22_CracklingFire_GlassBreaking_WaterDrops', '']


### Dataframe com as Classes Combinadas

In [5]:
dsInfo.reset_index(drop=True)

,Diretorio,Evento
0,BoltAction22,Disparo Bolt
1,CracklingFire,Incêndio
2,GlassBreaking,Vidro Quebrando
3,WaterDrops,Água Pingando
4,BoltAction22_CracklingFire,
5,BoltAction22_GlassBreaking,
6,BoltAction22_WaterDrops,
7,CracklingFire_GlassBreaking,
8,CracklingFire_WaterDrops,
9,GlassBreaking_WaterDrops,


### Carregamento dos Arquivos de Áudios

In [6]:
count = 0
tamMaiorAudio =0
qtdArqSubDir = list()
allArqAudio = list()

print('inicio')
inicio = time.time()

for subdir in dsInfo['Diretorio']:
    dirPath = Path(dsPath+'\\Audios\\'+subdir)
    arqObjectsPath = list(dirPath.glob('**/*.wav'))
    qtdArqSubDir.append(len(arqObjectsPath))
    
    for x in arqObjectsPath:
        count = count + 1
        audSignal, audSR = librosa.load(x, sr=44100)
        allArqAudio.append([subdir, x, audSignal.tolist(), len(audSignal)])
        
print(count)
fim = time.time()
print('fim - duração {0}'.format(fim-inicio))
    
dsInfo['QtdArq'] = qtdArqSubDir

inicio
1695
fim - duração 4.867934465408325


### Conversão da Lista de Áudios em Dataframe e Total por Tipo

In [7]:
allArqAudioDF = pd.DataFrame(allArqAudio,columns=['diretorio','path','audio','tamanho'])
allArqAudioDF.groupby(['diretorio']).count()

,path,audio,tamanho
diretorio,,,
BoltAction22,142,142,142
BoltAction22_CracklingFire,102,102,102
BoltAction22_CracklingFire_GlassBreaking,102,102,102
BoltAction22_CracklingFire_GlassBreaking_WaterDrops,102,102,102
BoltAction22_CracklingFire_WaterDrops,102,102,102
BoltAction22_GlassBreaking,142,142,142
BoltAction22_GlassBreaking_WaterDrops,111,111,111
BoltAction22_WaterDrops,111,111,111
CracklingFire,102,102,102


### Declaração da Função Geradora do Espectrograma Mel

In [8]:
def geraSpectrograma(audioSignal, sampleRate):
    mel_spectrogram = librosa.feature.melspectrogram(audioSignal, sr=sampleRate, n_fft=FFT_FRAME_SIZE, 
                                                      hop_length=HOP_LENGTH, n_mels=10)
    log_mel_spectrogram = librosa.power_to_db(mel_spectrogram)
    
    return log_mel_spectrogram

### Declaração da Função que Salva o Espectrograma em Arquivo

In [9]:
def salvaSpectrograma(nomeSemExt, spectro):
    arqSpectro = str(nomeSemExt)
    arqSpectro = arqSpectro.replace('Audios','Spectrogramas')
    arqSpectro = arqSpectro + '.npy'

    tempPath = Path(arqSpectro)
    if not tempPath.parents[0].exists():
        tempPath.parents[0].mkdir(parents=True)
    np.save(arqSpectro, spectro) 

### Declaração da Função que Salva o Novo Segmento de Áudio em Arquivo

In [10]:
def salvaAudioSegmento(nomeSemExt, segment, rate):
    arqAudio = str(nomeSemExt)
    arqAudio = arqAudio + '.wav'

    os.makedirs(os.path.dirname(arqAudio), exist_ok=True)
    with sf.SoundFile(arqAudio, 'w', samplerate=rate, channels=1, subtype='PCM_16') as f:
        f.write(segment)

### Declaração da Função que Insere Ruído Aleatório em Áudio Recebido

In [11]:
def inputNoise(audio, factor):
    noise = np.random.random_sample(len(audio))
    noise =  noise * factor
    
    audio_noise = audio + (audio * noise)
    audio_noise = audio_noise.astype(type(audio[0]))
    
    return audio_noise

### Declaração da Função Gera Novos Áudios com Ruído, Gera Espectrograma e Salva Arquivos

In [12]:
def geraRuidoAudios(qtdAmostras, dirInfo, arqAudioDF):
    qtdNovasAmostras = int(qtdAmostras)

    dirPath = str(dirInfo)
    arqNameBase = dirPath[0:dirPath.rindex('.')] + '_'
                         
    for x in range(0, qtdNovasAmostras):
        arqNome = arqNameBase+str(x+1)+'nse'
        audioNoised = inputNoise(arqAudioDF, 0.05)        
        spectrograma = geraSpectrograma(audioNoised, 44100)
        salvaSpectrograma(arqNome, spectrograma)
        salvaAudioSegmento(arqNome, audioNoised, 44100)            

### Loop no Dataframe de Áudios para Geração de Áudios Sintéticos

In [13]:
#Data Augmentation usando injeção de ruídos

print('inicio')
inicio = time.time()

for audioItem in allArqAudioDF.values.tolist():
    geraRuidoAudios(2, audioItem[1], audioItem[2])
    
fim = time.time()
print('fim - duração {0}'.format(fim-inicio))

inicio
fim - duração 56.488646268844604
